<a href="https://www.kaggle.com/code/ayushsavaliya2002/sentiment-analysis?scriptVersionId=177234492" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
import zipfile
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.parsing.preprocessing import remove_stopwords
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords')

2024-05-12 14:42:33.226975: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-12 14:42:33.227123: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-12 14:42:33.403734: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
df = pd.read_csv('/kaggle/input/sentiment140/training.1600000.processed.noemoticon.csv',names=['sen','id','date','flag','user','description'],encoding='latin-1' )

In [3]:
df = df.sample(frac = 1)
df = df.iloc[:50000]

In [4]:
df.head()

,sen,id,date,flag,user,description
574431,0,2210420903,Wed Jun 17 11:33:52 PDT 2009,NO_QUERY,JoshuaChr,@zoot_woman Boo! No US shows? #heartbroken
485992,0,2181351280,Mon Jun 15 11:42:37 PDT 2009,NO_QUERY,soniakd,i keep dreaming about people throwing jars of ...
1453618,4,2063251346,Sun Jun 07 01:55:37 PDT 2009,NO_QUERY,holisticguru,@klavoie morning kate! are you going to the ra...
1080254,4,1968215102,Fri May 29 20:59:23 PDT 2009,NO_QUERY,Dancer4Life13,@kayte_girll I'm going to bed now. Ttyl. Thx f...
566240,0,2206775337,Wed Jun 17 07:02:16 PDT 2009,NO_QUERY,Sturta,@ChadLeach I tried to update and got an error....


In [5]:
df = df.drop(['id','date','flag', 'user'], axis = 1)

In [6]:
import re

def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    text_without_urls = url_pattern.sub('', text)

    return text_without_urls

df['description'] = df['description'].apply(remove_urls)

In [7]:
df['description'] = df['description'].str.lower()
df['description'] = df['description'].astype(str)

In [8]:
def remove_punk(text):
  return text.translate(str.maketrans('', '', string.punctuation))

df['description'] = df['description'].apply(remove_punk)
df['description'] = df['description'].apply(remove_stopwords)

In [9]:
X_train,X_test,y_train,y_test = train_test_split(df['description'],df['sen'],test_size=0.2,random_state=1)

In [10]:
y_train = y_train/4
y_test = y_test/4

In [11]:
y_train.value_counts()

sen
0.0    20001
1.0    19999
Name: count, dtype: int64

In [12]:
tfidf = TfidfVectorizer()

X_train = tfidf.fit_transform(X_train).toarray()
X_test = tfidf.transform(X_test)

In [13]:
X_train.shape

(40000, 53693)

In [14]:
model = Sequential()

model.add(Dense(3,'relu'))
model.add(Dense(1,'sigmoid'))

In [15]:
es = tf.keras.callbacks.EarlyStopping(
    monitor='val_accuracy',
    verbose=1,
    restore_best_weights=True,
    patience=2
)

In [16]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [17]:
hist = model.fit(x=X_train, y=y_train, validation_split=.2, batch_size= 8, epochs=10, callbacks=es)

Epoch 1/10
4000/4000 ━━━━━━━━━━━━━━━━━━━━ 21s 5ms/step - accuracy: 0.6241 - loss: 0.6524 - val_accuracy: 0.7429 - val_loss: 0.5435
Epoch 2/10
4000/4000 ━━━━━━━━━━━━━━━━━━━━ 17s 4ms/step - accuracy: 0.8334 - loss: 0.4493 - val_accuracy: 0.7516 - val_loss: 0.5101
Epoch 3/10
4000/4000 ━━━━━━━━━━━━━━━━━━━━ 18s 4ms/step - accuracy: 0.8828 - loss: 0.3341 - val_accuracy: 0.7448 - val_loss: 0.5229
Epoch 4/10
4000/4000 ━━━━━━━━━━━━━━━━━━━━ 17s 4ms/step - accuracy: 0.9155 - loss: 0.2532 - val_accuracy: 0.7387 - val_loss: 0.5628
Epoch 4: early stopping
Restoring model weights from the end of the best epoch: 2.


In [18]:
model.evaluate(X_test,y_test)

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7359 - loss: 0.5315


[0.5295940041542053, 0.7376000285148621]

In [19]:
model.save('sent-model.h5')

In [20]:
def prediction(text):
    text = remove_stopwords(remove_urls(text).lower())
    text = tfidf.transform([text]).toarray()
    pred = model.predict(text)
    print('positive' if pred>.5 else 'negative')

In [21]:
prediction("A very good product!!")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
positive


In [22]:
prediction("A very bad product")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
negative


In [23]:
prediction("Value for money!!")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
positive


In [24]:
prediction("Too Expensive")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
negative
